# Why Animate

Here are some reasons you may want to use ahlive to animate your data to life!

### emphasize outliers

By having dynamic limits, outliers can be easily highlighted.

In [ ]:
import ahlive as ah
import pandas as pd

df = ah.open_dataset("owid_co2_concentrations_over_the_long_term_scripps")
df = pd.concat([df.loc[:1850][::6], df.loc[1850:][::2]])

ah.DataFrame(
    df,
    xs="year",
    ys="co₂_concentration_scripps",
    state_labels="co₂_concentration_scripps",
    xlims="fixed",
    ylims="explore",
    ymargins=0.1
).config(
    "state",
    suffix=" PPM",
    format=".0f"
).render()

### add dimension

Data can be mapped to color, and also time!

In [ ]:
import ahlive as ah
import xarray as xr

ds = xr.tutorial.open_dataset("air_temperature")
ds["month"] = ds["time"].dt.strftime("%m - %b")
ds = ds.groupby("month").mean()

tmpf_kcmi_inline = ds["air"].sel(lon=-88.5 + 360, lat=40, method="nearest")

ah_ds = ah.Dataset(
    ds,
    xs="lon",
    ys="lat",
    cs="air",
    state_labels="month",
    title="2013-2014 Monthly Averaged Temperatures",
    projection="LambertConformal",
    coastline=True,
    durations=1
).reference(
    x0s=[-88.5] * 12,  # help wanted to simplify this method!
    y0s=[40] * 12,
    inline_labels=tmpf_kcmi_inline
).config(
    "ref_inline", suffix="°F"
).render()

<div class="alert alert-warning">

Adding extra dimensionality to a visualization does not equate to better! Remember, the primary goal of a data visualization is to communicate an aspect of the data effectively.

</div>

### show gradually

Rather than showing everything at once and overwhelming the audience, a little story can be expressed!

In [ ]:
import ahlive as ah

df = ah.open_dataset(
    label="iem_asos",
    stn=["KDEN"],
    ini="2019-10-09",
    end="2019-10-10",
    tz="mst",
    data="tmpf",
    verbose=True
)

ah.DataFrame(
    df,
    xs="valid",
    ys="tmpf",
    xlims="fixed",
    ylim0s="explore",
    ymargins=0.1,
    title="Temperature at Denver Airport",
).remark(
    "High of 82°F",
    ys=82,
    durations=3
).remark(
    "Low of 30°F within 8 hours!",
    ys=30,
    first=True,
    durations=3,
).config(
    "remark_inline",
    ha="right"
).render()

### engage audience

Sometimes, it's just more fun!

In [ ]:
import ahlive as ah

life_df = ah.open_dataset("gapminder_life_expectancy")
gdp_df = ah.open_dataset("gapminder_income")
df = life_df.merge(gdp_df, on=["geo", "time"])
df = df.loc[
    (df["geo"].isin(df["geo"].sample(5).unique())) &
    (df["time"].isin([1880, 2020]))
]
df["geo"] = df["geo"].str.upper()
df_avg = df.groupby('time').mean()

ah.DataFrame(
    df,
    xs="life_expectancy",
    ys="income",
    label="geo",
    title="Gapminder",
    inline_labels="geo",
    state_labels="time",
    revert="boomerang",
    chart="scatter",
    legend=False,
    durations=1,
    figsize=(10, 5)
).reference(
    x0s=df_avg["life_expectancy"],
    inline_labels=df_avg["life_expectancy"],
    inline_locs=df_avg["income"].median(),
    alpha=0.2
).reference(
    y0s=df_avg["income"],
    inline_labels=df_avg["income"],
    inline_locs=df_avg["life_expectancy"].median(),
    alpha=0.2
).render()

### display chronologically

What better way to display chronological data than to animate!

In [ ]:
import ahlive as ah

df = ah.open_dataset("gapminder_population")
df = df.pivot("time", "geo", "population").diff().reset_index().melt(
    "time", value_name="population_change")
df = df.loc[df["time"].between(2000, 2020)]
df["geo"] = df["geo"].str.upper()
df["population_change"] /= 1e6

ah.DataFrame(
    df,
    xs="geo",
    ys="population_change",
    title="Yearly Increases in Population",
    ylabel="[million]",
    label="geo",
    state_labels="time",
    inline_labels="population_change",
    chart="bar",
    preset="race",
).render()